[View in Colaboratory](https://colab.research.google.com/github/shivangpandya/nueral-network/blob/master/fruitsclassifier.ipynb)

In [0]:
!mkdir data

In [3]:
cd data

/content/data


In [4]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/grapes.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy

--2018-08-21 12:43:41--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 2607:f8b0:4001:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113462128 (108M) [application/octet-stream]
Saving to: ‘apple.npy’

apple.npy           100%[===================>] 108.21M   107MB/s    in 1.0s    

2018-08-21 12:43:42 (107 MB/s) - ‘apple.npy’ saved [113462128/113462128]

--2018-08-21 12:43:43--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.200.128, 2607:f8b0:4001:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98055744 (94M) [application/octet-stream]
Saving to: ‘pineapple.n

In [6]:
cd ..

/content


In [0]:
from sklearn.model_selection import train_test_split as tts
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
from PIL import Image

In [0]:
#define some constants
N_FRUITS = 4
FRUITS = { 0:"Apple" , 1:"Banana" , 2:"Grape" , 3:"Pineapple"}

#Number of samples to take in each class
N = 5000

#Some other constants
N_EPOCHS = 10

# data files in the same order as defiend in fruits
files = {"apple.npy" , "banana.npy" , "grapes.npy" , "pineapple.npy"}


In [0]:
#Function to  load .npy files from disk ad return as numpy arrays
def load(dir,reshaped,files):
  data = []
  for file in files:
    f= np.load(dir+file)
    if reshaped:
      new_f = []
      for i in range(len(f)):
        x = np.reshape(f[i],(28,28))
        x = np.expand_dims(x, axis = 0)
        x = np.reshape(f[i],(28,28,1))
      f = new_f
    data.append(f)
  return data
      
 #Takes a list or a list of lists and returns its normalized form
def normalise(data):
  return np.interp(data,[0,255],[-1,1])

#Takes a list or a list of lists and returns its normalized form
def denormalise(data):
  return np.interp(data , [-1, 1], [0, 255])

#Visualise 2D Array as image
def visualize(array):
  array = np.reshape(array,(28,28))
  img = Image.fromarray(array)
  return array

#Limit elements from each array upto n elements and returns a single list
def set_limit(arrays , n):
  new = []
  for array in arrays:
    i=0
    for item in array:
      if i==n:
        break
      new.append(item)
      i += 1
  return new

#Make labels from 0 to N1 , each repeated N2 times
def make_labels(N1,N2):
  labels = []
  for i in range(N1):
    labels += [i] * N2
  return labels      

In [0]:
fruits = load("data/", False, ['apple.npy'])

In [0]:
visualize(fruits[0][0])

In [33]:
#Second argument is true for reshaping the image to 28 X 28 form. A conv net expects this format
fruits = load("data/",True,files)

#second argument is False because we don't need to reshape the image. An MLP net expects this format.
#fruits = load("data/", False, files)

#Limit number of samples in each class to N
fruits  = set_limit(fruits , N)

#Normalise the values
fruits = map(normalise,fruits)

#Define the labels
labels = make_labels(N_FRUITS , N)

#Prepare the data
x_train, x_test, y_train, y_test = tts(fruits, labels, test_size=0.05)

#One hot encoding-make standard list into another list
Y_train = np_utils.to_categorical(y_train,N_FRUITS)
Y_test = np_utils.to_cateogical(y_test,N_FRUITS)

TypeError: ignored

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(N_FRUITS, activation='softmax'))

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# train
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=N_EPOCHS)

print("Training complete")

print("Evaluating model")
preds = model.predict(np.array(x_test))

score = 0
for i in range(len(preds)):
    if np.argmax(preds[i]) == y_test[i]:
        score += 1

print("Accuracy: ", ((score + 0.0) / len(preds)) * 100)


model.save("fruits"+ ".h5")
print("Model saved")

NameError: ignored